In [1]:
import json
from datasets import Dataset, load_dataset

# Load your data
train_data = load_dataset('json', data_files='train_dataset.json')
test_data = load_dataset('json', data_files='test_dataset.json')

/home/nanzera/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from transformers import AutoTokenizer
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2-7B",
    max_seq_length=512,
    load_in_4bit=True,
    dtype=None,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"], 
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

==((====))==  Unsloth 2024.8: Fast Qwen2 patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti SUPER. Max memory: 15.992 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.25. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 28 layers with 0 QKV layers, 28 O layers and 28 MLP layers.


In [3]:
def format_conversation(example):
    conversation = f"Human: {example['anchor']}?\n\nAssistant: {example['positive']}{tokenizer.eos_token}"
    return {"text": conversation}

train_data = train_data.map(format_conversation)
test_data = test_data.map(format_conversation)

# If train_data and test_data are DatasetDict objects, access the 'train' split
if isinstance(train_data, dict) and 'train' in train_data:
    train_data = train_data['train']
if isinstance(test_data, dict) and 'train' in test_data:
    test_data = test_data['train']

# Verify the format
print("Sample from train_data:")
print(train_data[0]['text'])
print("\nSample from test_data:")
print(test_data[0]['text'])

Map: 100%|██████████| 296/296 [00:00<00:00, 15904.41 examples/s]

Sample from train_data:
Human: 什么是 过电压??

Assistant: 超过规定限值的电压。<|endoftext|>

Sample from test_data:
Human: 什么是 总保护??

Assistant: 安装在配电台区低压侧的第一级剩余电流动作保护器，亦称总保。<|endoftext|>


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    dataset_text_field="text",
    max_seq_length=512,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-4,
        lr_scheduler_type="linear",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        warmup_steps=10,
        output_dir="output",
        seed=0,
    ),
)

trainer.train()

Generating train split: 231 examples [00:00, 918.53 examples/s]
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 231 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 29
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,2.187800
2,2.055900
3,2.208000
4,2.065800
5,2.076100
6,2.097900
7,1.921500
8,1.849000
9,1.892900
10,1.912300


TrainOutput(global_step=29, training_loss=1.829147560843106, metrics={'train_runtime': 95.8593, 'train_samples_per_second': 2.41, 'train_steps_per_second': 0.303, 'total_flos': 5046185567453184.0, 'train_loss': 1.829147560843106, 'epoch': 1.0})

In [10]:
trainer.model.save_pretrained("./fine_tuned_qwen2_7b_sft")
tokenizer.save_pretrained("./fine_tuned_qwen2_7b_sft")

quant_methods = ["q2_k", "q3_k_m", "q4_k_m", "q5_k_m", "q6_k", "q8_0"]
for quant in quant_methods:
    model.save_pretrained_gguf("gguf", tokenizer, quant)

AttributeError: 'NoneType' object has no attribute 'save_pretrained_gguf'

In [ ]:
from transformers import pipeline

# Merge the LoRA weights with the base model
from peft import AutoPeftModelForCausalLM
import torch

merged_model = AutoPeftModelForCausalLM.from_pretrained(
    "./fine_tuned_qwen2_sft",
    device_map="auto",
    torch_dtype=torch.float16,
)
merged_model = merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("./fine_tuned_qwen2_sft_merged")
tokenizer.save_pretrained("./fine_tuned_qwen2_sft_merged")

# Create a pipeline for the merged model
generator = pipeline("text-generation", model="./fine_tuned_qwen2_sft_merged", tokenizer="./fine_tuned_qwen2_sft_merged")

# Test with a sample term
test_term = "water"
messages = [
    {"role": "human", "content": f"Explain the term: {test_term}"}
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False)
generated_text = generator(prompt, max_length=200, num_return_sequences=1)

print(generated_text[0]['generated_text'])